In [58]:
# sprawozdanie na za 2 tygodnie( 23 marca do pory zajęć)
# zerowanie w sposób jawny wyrazów w macierzy
# partial pivoting - szukanie elementu wiodącego w kolumnie
# full pivoting - j.w. tylko że również w wierszach
# scaling - scalowanie każdego wiersza macierzy przez rząd wielkości n.p
# bez pivotingu, scaling można (pivot w domu)
# do 3 ważny rysunek. Im lepszy tym lepiej
# ryswanie grafów
# sprawdzanie czy sumy prądów na węzłach się zgadzają
# skomplikowane grafów
# układ równań normalnych
# Kirchoff na lepszą ocenę
import numpy as np
import time

#### Metoda Gaussa-Jordana

In [60]:

def Gauss_Jordan(matrix_A, matrix_b):
    matrix = np.column_stack((matrix_A, matrix_b))
    
    for p in range(np.size(matrix, 0)):
        pivot = matrix[p][p]
        for row in range(0, np.size(matrix,0)):
            if row != p and pivot!=0:
                row_sub = -matrix[p, :]*(matrix[row][p]/pivot)
                matrix[row, :] = matrix[row,:] + row_sub
                if row > p:
                    matrix[row, 0:p] = 0
                else:
                    matrix[row, p:np.size(matrix,0)-1] = 0
    return matrix


matrixA = np.random.rand(500, 500)
matrixb = np.random.rand(500)
start = time.time()
matrix = Gauss_Jordan(matrixA, matrixb)
end = time.time()

start2 = time.time()
matrix_2 =  np.linalg.solve(matrixA, matrixb)
end2 = time.time()
elapsed2 = end2-start2

elapsed = end-start
print(elapsed)
print(elapsed2)
print(matrix)

3.6928727626800537
0.348330020904541
[[ 8.21761197e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -3.39283626e-01]
 [ 0.00000000e+00  4.71688574e-01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  4.07726036e-01]
 [ 0.00000000e+00  0.00000000e+00 -6.40911602e-02 ...  0.00000000e+00
   0.00000000e+00 -1.21321505e-02]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -9.23091853e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.27880154e+02
   0.00000000e+00 -5.15936145e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
  -1.27544859e+01 -1.63112394e+01]]
